# Figures

Here, additional figures are generated for the neuefische GmbH Data Science Bootcamp Cologne graduation event.

## Imports

In [ ]:
import os
import pathlib

import pandas as pd
import numpy as np
import cv2 as cv
import imageio
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

import skimage
from skimage import io
from skimage import color
from skimage import filters
from skimage.exposure import equalize_adapthist
from skimage.exposure import adjust_gamma

from PIL import Image

## Colors

In [ ]:
fontcolor = "#EEEEEE"
nf_orange = "#FF5A36"

## Specify which predictions to use and create corresponding figure folder

In [ ]:
pred_dir_pick = "U-net-from-scratch_astroS20E50_cortS60E10_shsy5yS24E25" # Folder from which predictions are taken

curr_dir = os.getcwd()
parent_dir = pathlib.Path(curr_dir).parents[1]

pred_dir = f"{parent_dir}/data/data_preprocessed/mask_predicted/{pred_dir_pick}/"

figures_dir = f"{parent_dir}/notebooks/U-net/figures"
fig_dir = f"{parent_dir}/notebooks/U-net/figures/{pred_dir_pick}"

try:
    os.mkdir(figures_dir)
except:
    print("Figure directory already exists")
    
try:
    os.mkdir(fig_dir)
except:
    print("Figure directory already exists")

In [ ]:
# Path to the original images
img_path = f"{parent_dir}/data/data_original/train/"

# Path to the original masks
msk_path = f"{parent_dir}/data/data_preprocessed/masks_used_for_model/"

# Path to the groundtruth masks (to get the astro masks provided via Kaggle)
msk_kaggle_path = f"{parent_dir}/data/data_preprocessed/mask_groundtruth/"

# Path to the predicted masks
prd_path = f"{parent_dir}/data/data_preprocessed/mask_predicted/{pred_dir_pick}/"

## Load the IoU data

In [ ]:
df_iou = pd.read_csv(f"{pred_dir}IoU.csv")
df_iou.head()

## Figure: IoU histogram
**Note:** Figure may be displayed with black and white background in this notebook, but the export is fully transparent

In [ ]:
dataset = "test"

df2 = df_iou[df_iou["dataset"]==dataset]
uniq_cell_type = df2["cell_type"].unique()
uniq_cell_type.sort()

fig, axes = plt.subplots(1, 3)
fig.set_figheight(4)
fig.set_figwidth(12)
fig.suptitle(f"IoU distributions per cell type ({dataset} data only)", fontsize=20)

plt.rcParams.update({'text.color' : fontcolor,
                     'axes.labelcolor' : fontcolor,
                     'xtick.color' : fontcolor,
                     'ytick.color' : fontcolor})

for i, cell_type in enumerate(uniq_cell_type):
    df3 = df2[df2["cell_type"]==cell_type]

    g = sns.histplot(ax = axes[i], data = df3, x = "IoU", binwidth = 0.05,
                    edgecolor = fontcolor, color = nf_orange)

    # Mean IoU
    mean_iou = np.mean(df3["IoU"])

    if dataset != "test":
        axes[i].set(xlim = (-0.01, 1), ylim = (0, 60))
        axes[i].vlines(mean_iou, 0, 55, colors=fontcolor, linestyles='dashed') 
    else:
        axes[i].set(xlim = (-0.01, 1), ylim = (0, 25))
        axes[i].vlines(mean_iou, 0, 22, colors=fontcolor, linestyles='dashed') 

    trans = axes[i].get_xaxis_transform()
    axes[i].text(mean_iou+0.02,0.875,f"Mean: {round(mean_iou,2)}", transform=trans, fontsize=14)

    axes[i].set_title(cell_type, fontsize=16)
    axes[i].set_xlabel('IoU', fontsize=14)
    axes[i].set_ylabel('N images', fontsize=14)
    axes[i].spines['left'].set_color(fontcolor)
    axes[i].spines['bottom'].set_color(fontcolor) 

    sns.despine(top = True, right = True)

    plt.tight_layout()
    plt.savefig(f"{fig_dir}/pres_mean_IoU_{dataset}.png", dpi=300, transparent=True)

## Define functions for image transformation

In [ ]:
def make_IoU_img(msk, prd):
    prd = np.expand_dims(prd, axis = 2)
    msk = np.expand_dims(msk, axis = 2)
    
    r = ((prd!=0) | (msk!=0)).astype(dtype = np.uint8) * 255
    g = ((prd!=0) & (msk!=0)).astype(dtype = np.uint8) * 255
    b = np.zeros(r.shape)

    img_iou = np.concatenate((r, g, b), axis = 2)
    
    return img_iou


def make_mask(img, transparent_background = False, use_for_overlay = False):
        g = np.expand_dims(img!=0, axis = 2).astype(dtype = np.uint8) * 255
        
        if use_for_overlay: # segmentation highlighted in green
            r = np.zeros(g.shape)
            b = np.zeros(g.shape)
        else: # segmentation highlighted in white
            r = np.expand_dims(img!=0, axis = 2).astype(dtype = np.uint8) * 255
            b = np.expand_dims(img!=0, axis = 2).astype(dtype = np.uint8) * 255
        
        img = np.concatenate((r, g, b), axis = 2)

        # Use Pillow to make every black pixel fully transparent
        if transparent_background:
            img = Image.fromarray(img.astype(np.uint8))
            img = img.convert("RGBA")

            datas = img.getdata()
            newData = []
            for item in datas:
                if item[0] == 0 and item[1] == 0 and item[2] == 0:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(item)

            img.putdata(newData)

        return img


def enhance_image(img):
    img = equalize_adapthist(img, clip_limit=0.025)
    img = adjust_gamma(img, gamma=0.6,gain=1)
    
    return img


def img_slicer(img):
    HEIGHT = img.shape[0]
    WIDTH  = img.shape[1]
    
    upper = range(0, int(HEIGHT/2))
    lower = range(int(HEIGHT/2), HEIGHT)
    
    left  = range(0, int(WIDTH/2))
    right = range(int(WIDTH/2), WIDTH)
    
    img_a = img[upper,:][:,left]
    img_b = img[upper,:][:,right]
    img_c = img[lower,:][:,left]
    img_d = img[lower,:][:,right]
    
    return img_b # For now, hard-coded to return the upper right portion
    


def create_images(img_id, img_path, msk_path, prd_path, use_slice = False):
    # Store the following in a dictionary for visualization:
    # - enhanced image (more contrast and brightness)
    # - original/predicted mask with black background and bright cell segments
    # - original/predicted mask with transparent background for image-mask overlay
    # - IoU visualization
    
    # Read image, original mask and prediction
    img = imageio.imread(f"{img_path}{img_id}.png")
    msk = imageio.imread(f"{msk_path}{img_id}_mask.png")
    prd = imageio.imread(f"{prd_path}masks/{img_id}_pred.png")
    
    # If only a single quadrant of the image is used:
    if use_slice:
        img = img_slicer(img)
        msk = img_slicer(msk)
        prd = img_slicer(prd)
        

    # Enhance the image's contrast and brightness and store in dictionary
    img_dict = {"image": enhance_image(img)}
    # img_dict = {"image": img} # show the unenhanced image instead
    
    # Convert prediction and mask colors and store without/with transparent background
    img_dict["msk_orig"] = make_mask(msk)
    img_dict["msk_pred"] = make_mask(prd)
    img_dict["msk_orig_trans"] = make_mask(msk, transparent_background = True, use_for_overlay = True)
    img_dict["msk_pred_trans"] = make_mask(prd, transparent_background = True, use_for_overlay = True)
    
    # Create and store IoU
    img_dict["IoU"] = make_IoU_img(msk, prd)
    
    return img_dict

## Figure: Enhanced image with predicted mask overlay

In [ ]:
# To be visualized image IDs
img_ids = ["0140b3c8f445", "6b165d790e33", "db8bc8f09776"]

In [ ]:
for img_id in img_ids:
    
    # Read IoU and cell type
    IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
    cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
    
    # Read and convert images and masks
    img_dict = create_images(img_id, img_path, msk_path, prd_path)
    
    # Plot original image with mask overlay
    fig = plt.figure(figsize = (10, 10))
    plt.imshow(img_dict["image"], cmap = "gray")
    plt.imshow(img_dict["msk_pred_trans"], alpha = 0.3)
    plt.axis('off')
    plt.savefig(f"{fig_dir}/pred_overlay_{cell_type}_{img_id}.png", dpi=500)

## Figure: Sliced/unsliced examples without mask overlay

In [ ]:
# To be visualized image IDs
img_ids = ["0140b3c8f445", "6b165d790e33", "db8bc8f09776"]

for use_slice in [True, False]:
    for img_id in img_ids:
        
        # Read IoU and cell type
        IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
        cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
        
        # Read and convert images and masks
        img_dict = create_images(img_id, img_path, msk_path, prd_path, use_slice = use_slice)
        
        # Plot original image with mask overlay
        fig = plt.figure(figsize = (10, 10))
        plt.imshow(img_dict["image"], cmap = "gray")
        plt.axis('off')
        
        if use_slice:
            plt.savefig(f"{fig_dir}/img_b_only_{cell_type}_{img_id}.png", dpi=500)
        else:
            plt.savefig(f"{fig_dir}/img_only_{cell_type}_{img_id}.png", dpi=500)

## Figure: Un(sliced) masks only

In [ ]:
# To be visualized image IDs
img_ids = ["0140b3c8f445", "6b165d790e33", "db8bc8f09776"]

for use_slice in [True, False]:
    for img_id in img_ids:
        
        # Read IoU and cell type
        IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
        cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
        
        # Read and convert images and masks
        img_dict = create_images(img_id, img_path, msk_path, prd_path, use_slice = use_slice)
        
        # Plot original image with mask overlay
        fig = plt.figure(figsize = (10, 10))
        plt.imshow(img_dict["msk_pred"])
        plt.axis('off')
        
        if use_slice:
            plt.savefig(f"{fig_dir}/msk_b_only_{cell_type}_{img_id}.png", dpi=500)
        else:
            plt.savefig(f"{fig_dir}/msk_only_{cell_type}_{img_id}.png", dpi=500)

## Figure: Sliced examples with prediction

In [ ]:
# To be visualized image IDs
img_ids = ["0140b3c8f445", "6b165d790e33", "db8bc8f09776"]

for img_id in img_ids:
        
    # Read IoU and cell type
    IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
    cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
    
    # Read and convert images and masks
    img_dict = create_images(img_id, img_path, msk_path, prd_path, use_slice = True)
    
    # Plot original image with mask overlay
    fig = plt.figure(figsize = (10, 10))
    plt.imshow(img_dict["image"], cmap = "gray")
    plt.imshow(img_dict["msk_pred_trans"], alpha = 0.3)
    plt.axis('off')
    
    plt.savefig(f"{fig_dir}/img_b_pred_overlay_{cell_type}_{img_id}.png", dpi=500)


## Figure: Enhanced image with CG mask overlay (astro only)

In [ ]:
img_id = "0140b3c8f445"

# Read IoU and cell type
IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]

# Read and convert images and masks
img_dict = create_images(img_id, img_path, msk_path, prd_path)

# Plot original image with mask overlay
fig = plt.figure(figsize = (10, 10))
plt.imshow(img_dict["image"], cmap = "gray")
plt.imshow(img_dict["msk_orig_trans"], alpha = 0.3)
plt.axis('off')
plt.savefig(f"{fig_dir}/mask_cg_{cell_type}_{img_id}.png", dpi=500)

## Figure: Enhanced image with original Kaggle mask overlay (astro only)

In [ ]:
img_id = "0140b3c8f445"

# Read IoU and cell type
IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]

# Read and convert images and masks
img_dict = create_images(img_id, img_path, msk_kaggle_path, prd_path)

# Plot original image with mask overlay
fig = plt.figure(figsize = (10, 10))
plt.imshow(img_dict["image"], cmap = "gray")
plt.imshow(img_dict["msk_orig_trans"], alpha = 0.3)
plt.axis('off')
plt.savefig(f"{fig_dir}/mask_kaggle_{cell_type}_{img_id}.png", dpi=500)

## Figure: K-means cluster + Gauss filter example

In [ ]:
km_img    = imageio.imread("../run_once_preprocessing/kmeans_ex_image.png")
km_kmeans = imageio.imread("../run_once_preprocessing/kmeans_ex_kmeans.png")
km_gauss  = imageio.imread("../run_once_preprocessing/kmeans_ex_gauss.png")
km_pred   = imageio.imread("../run_once_preprocessing/kmeans_ex_mask.png")

# km_img = enhance_image(km_img)
km_pred = make_mask(km_pred)

In [ ]:
fig, ax = plt.subplots(1, 4)
fig.set_figheight(5)
fig.set_figwidth(15)

ax[0].imshow(adjust_gamma(km_img, gamma=0.6,gain=1), cmap='gray')
ax[1].imshow(adjust_gamma(km_kmeans, gamma=0.6,gain=1), cmap='gray')
ax[2].imshow(adjust_gamma(km_gauss, gamma=0.6,gain=1), cmap='gray')
ax[3].imshow(km_pred)


for i in range(0,4):
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.tight_layout()
plt.savefig("figures/kmeans_example.png", dpi=300)

## Figure: IoU example

In [ ]:
for img_id in img_ids:
    IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
    cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
    
    # Read and convert images and masks
    img_dict = create_images(img_id, img_path, msk_path, prd_path)
    
    plt.rcParams.update({'text.color' : "#eeeeee",
                     'axes.labelcolor' : "#eeeeee",
                        'xtick.color' : "#eeeeee",
                        'ytick.color' : "#eeeeee"})
    
    fig, ax = plt.subplots(1, 3)
    fig.set_figheight(3.3)
    fig.set_figwidth(10)
    fig.suptitle("Cell type: {}, intersection over union: {}".format(cell_type, round(IoU,2)),
                 fontsize = 16)
    
    ax[0].imshow(img_dict["msk_pred"])
    ax[0].set_title("Prediction")
    
    ax[1].imshow(img_dict["msk_orig"])
    ax[1].set_title("Original")
    
    ax[2].imshow(img_dict["IoU"])
    ax[2].set_title("Intersection over Union")
    
    for i in range(0,3):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        ax[i].set_xticks([])
        ax[i].set_yticks([])
    
    plt.tight_layout()
    plt.savefig(f"{fig_dir}/example_pred2_{cell_type}_{img_id}.png", dpi=300)

## Figure: Best/worst test IoU per cell type

In [ ]:
df2 = df_iou[df_iou["dataset"]=="test"]

uniq_cell_type = df2["cell_type"].unique()

img_ids_dict = dict()

for cell_type in uniq_cell_type:
    df3 = df2[df2["cell_type"]==cell_type]
    
    img_ids_dict[f"{cell_type}_worst"] = df3.loc[df3["IoU"]==np.min(df3["IoU"]), "id"].values[0]
    img_ids_dict[f"{cell_type}_best"]  = df3.loc[df3["IoU"]==np.max(df3["IoU"]), "id"].values[0]

In [ ]:
for img_type, img_id in img_ids_dict.items():
    
    IoU       = df_iou.loc[df_iou["id"]==img_id, "IoU"].values[0]
    cell_type = df_iou.loc[df_iou["id"]==img_id, "cell_type"].values[0]
    
    # Read and convert images and masks
    img_dict = create_images(img_id, img_path, msk_path, prd_path)
    
    plt.rcParams.update({'text.color' : "#eeeeee",
                     'axes.labelcolor' : "#eeeeee",
                        'xtick.color' : "#eeeeee",
                        'ytick.color' : "#eeeeee"})
    
    fig, ax = plt.subplots(1, 4)
    fig.set_figheight(2.8)
    fig.set_figwidth(10)
    fig.suptitle("Cell type: {}, intersection over union: {}".format(cell_type, round(IoU,2)),
                 fontsize = 16)
    
    ax[0].imshow(img_dict["image"], cmap = "gray")
    ax[0].set_title("Image")
    
    ax[1].imshow(img_dict["msk_pred"])
    ax[1].set_title("Prediction")
    
    ax[2].imshow(img_dict["msk_orig"])
    ax[2].set_title("Original")
    
    ax[3].imshow(img_dict["IoU"])
    ax[3].set_title("Intersection over Union")
    
    for i in range(0,4):
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        ax[i].set_xticks([])
        ax[i].set_yticks([])
    
    plt.tight_layout()
    plt.savefig(f"{fig_dir}/{img_type}_{img_id}.png", dpi=500)